In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import pywt as wt
import scipy.signal as sig
from scipy.io import loadmat
%matplotlib qt5

In [42]:
from cesium import datasets
eeg = datasets.fetch_andrzejak()
print(eeg.keys())

Loaded data from cached archive.


dict_keys(['times', 'measurements', 'classes', 'archive', 'header'])


In [13]:
from cesium import featurize
features_to_use = ["amplitude",
                   "percent_beyond_1_std",
                   "maximum",
                   "max_slope",
                   "median",
                   "median_absolute_deviation",
                   "percent_close_to_median",
                   "minimum",
                   "skew",
                   "std",
                   "weighted_average"]

fset_cesium = featurize.featurize_time_series(times=eeg["times"],
                                              values=eeg["measurements"],
                                              errors=None,# o que essa erros faz exatamente?
                                              features_to_use=features_to_use)


In [54]:
np.array(eeg['measurements']).shape
x = eeg['measurements'][0]

In [39]:
fset_cesium  # gera tabela com todas as features que eu selecionei

feature,amplitude,percent_beyond_1_std,maximum,max_slope,median,median_absolute_deviation,percent_close_to_median,minimum,skew,std,weighted_average
channel,0,0,0,0,0,0,0,0,0,0,0
0,143.5,0.162558,141.0,11107.796610,-4.0,28.0,0.505004,-146.0,0.032805,40.411000,-4.132048
1,211.5,0.145472,169.0,20653.559322,-51.0,32.0,0.640469,-254.0,-0.092715,48.812668,-52.444716
2,165.0,0.152307,184.0,13537.627119,13.0,31.0,0.515987,-146.0,-0.004100,47.144789,12.705150
3,171.5,0.152307,162.0,17008.813559,-4.0,31.0,0.541128,-181.0,0.063678,47.072316,-3.992433
4,170.0,0.152062,152.0,13016.949153,-18.0,29.0,0.566268,-188.0,0.142753,44.910958,-17.999268
5,151.5,0.154503,123.0,10934.237288,-28.0,29.0,0.524774,-180.0,0.002501,45.022715,-27.823041
6,162.0,0.156456,147.0,12843.389831,-13.0,30.0,0.531853,-177.0,0.003179,46.259433,-13.334635
7,126.5,0.156700,129.0,8851.525424,-5.0,22.0,0.565780,-124.0,-0.023597,32.342124,-5.653893
8,105.0,0.158409,117.0,7463.050847,9.0,21.0,0.491091,-93.0,-0.002426,31.478248,8.200879


In [14]:
# definindo features que não são padrão do cesium, e incluindo-as no código

import scipy.stats as stats

def mean_signal(t, m, e):
    return np.mean(m)

def std_signal(t, m, e):
    return np.std(m)

def mean_square_signal(t, m, e):
    return np.mean(m ** 2)

def abs_diffs_signal(t, m, e):
    return np.sum(np.abs(np.diff(m)))

def skew_signal(t, m, e):
    return stats.skew(m)

# criando um dict com as features que acabei de fazer
features = {
    'mean':mean_signal,
    'std':std_signal,
    'mean_square':mean_square_signal,
    'abs_diffs':abs_diffs_signal,
    'skew':skew_signal
}

feature_set_renato = featurize.featurize_time_series(times=eeg['times'], values=eeg['measurements'], errors=None, 
                                                     features_to_use=list(features.keys()), 
                                                     custom_functions=features)

In [45]:
feature_set_renato

feature,mean,std,mean_square,abs_diffs,skew
channel,0,0,0,0,0
0,-4.132048,40.411000,1650.122773,46948.0,0.032805
1,-52.444716,48.812668,5133.124725,61118.0,-0.092715
2,12.705150,47.144789,2384.051989,51269.0,-0.004100
3,-3.992433,47.072316,2231.742495,75014.0,0.063678
4,-17.999268,44.910958,2340.967781,52873.0,0.142753
5,-27.823041,45.022715,2801.166463,52734.0,0.002501
6,-13.334635,46.259433,2317.747620,63591.0,0.003179
7,-5.653893,32.342124,1077.979497,41753.0,-0.023597
8,8.200879,31.478248,1058.134489,32107.0,-0.002426


In [56]:
# usando DWT, separo cada canal em 5 sub-bandas diferentes, e entao aplico a featurização com base nessas features daí de cima

n_channels = 5

eeg['dwts'] = [wt.wavedec(m, wt.Wavelet('db2'),level=n_channels-1) for m in eeg['measurements']] # pq o n_channels-1 ?? 


feature_set_dwt = featurize.featurize_time_series(times=None, values=eeg['dwts'], errors=None, 
                                                     features_to_use=list(features.keys()), 
                                                     custom_functions=features)

In [59]:
feature_set_dwt.head()

feature        mean                                                  std  \
channel           0          1         2         3         4           0   
0        -15.366940   0.336819  1.153131 -0.076844  0.025117  110.795972   
1       -209.912645  -0.864731 -1.003853 -0.023409 -0.040904  119.131399   
2         50.014166   7.357674 -2.150601  0.119560  0.014293  134.720674   
3        -16.633104  -7.004795 -1.218739  0.230567  0.193031  106.501641   
4        -70.948590 -11.156691  2.050380  0.001822 -0.046372  125.311382   

feature                                                ...        abs_diffs  \
channel          1          2          3          4    ...                0   
0        74.475214  56.705140  20.636593   5.783827    ...     23757.729091   
1        99.378150  71.151964  28.663814   7.815785    ...     29516.417843   
2        89.277026  60.388719  22.369005   6.233646    ...     30009.952928   
3        92.857722  70.021354  33.816731  11.425655    ...     26702.794099   
4        83.786002  58.065014  23.565817   7.188402    ...     23226.201919   

feature                                                              skew  \
channel             1             2             3             4         0   
0        21317.896490  41041.500565  29713.786133  13817.939140 -0.115368   
1        27033.595932  49078.720445  40493.547836  17837.910960 -0.060414   
2        22457.556865  43031.294635  31717.331360  14122.514028  0.086005   
3        25137.020721  45772.823895  48653.500566  26150.592648  0.092523   
4        20707.505637  40802.678705  32278.513998  16716.475292  0.260220   

feature                                          
channel         1         2         3         4  
0       -0.243157  0.164417 -0.047230  0.042562  
1       -0.184601  0.047339 -0.026525  0.126670  
2       -0.189957 -0.317464 -0.015477 -0.032315  
3       -0.175761  0.029742 -0.139365  0.151099  
4       -0.243530 -0.031016  0.035825 -0.003741  

[5 rows x 25 columns]

In [1]:
# Aplicando o método nos sinais do Kolodziej

In [4]:
data_array = loadmat('5Hz/S1_5Hz.mat')

In [20]:
data_array['X'] = np.transpose(data_array['X'])

In [23]:
data_array['DWTs'] = [wt.wavedec(m, wt.Wavelet('db2'), level=4) for m in data_array['X']]

In [39]:
feature_S1_5hz = featurize.featurize_time_series(times=None, values=data_array['DWTs'], errors=None, 
                                                features_to_use=list(features.keys()), custom_functions=features)

In [40]:
feature_S1_5hz

feature          mean                                                          \
channel             0             1             2             3             4   
0        1.216456e-07 -3.348528e-07  1.290648e-09 -1.291080e-07  1.155442e-09   
1        2.153464e-09 -1.347627e-07  1.182121e-08 -1.016207e-07  2.148877e-09   
2       -5.241092e-08 -2.238879e-08  2.081835e-08 -7.939952e-08  1.993462e-09   
3        1.389420e-07 -4.270598e-07  5.653148e-08 -2.043970e-07  1.486208e-09   
4        2.589134e-07 -6.029324e-08  8.658779e-08 -1.348784e-07  1.561735e-09   
5        6.021368e-09  1.104145e-07 -7.423569e-10 -9.983687e-08  2.463392e-09   
6        6.059390e-08  4.910443e-08 -3.749144e-08 -5.853894e-08  2.193871e-09   
7        3.569105e-08  5.155674e-10 -1.944868e-08 -9.554143e-08  2.177105e-09   
8        1.293596e-07 -7.131700e-08 -1.164457e-07 -5.636677e-08  1.982091e-09   
9        2.283998e-07 -3.629001e-07  6.945214e-08 -1.098469e-07  1.361181e-09   
10       7.114122e-08  2.486384e-08  1.243335e-08 -8.431363e-08  1.977964e-09   
11       1.826431e-07 -1.212322e-07 -1.123820e-07 -9.277775e-08  1.519322e-09   
12       3.027902e-07 -1.995036e-07  8.647696e-09 -1.028205e-07  1.713462e-09   
13       2.151705e-07 -7.501421e-08 -5.134018e-08 -1.276093e-07  1.738697e-09   
14       1.292662e-07 -1.961649e-09  3.364882e-09 -1.500029e-07  1.761135e-09   
15       2.063322e-07  2.046131e-07  1.235262e-07 -1.921530e-07  1.537721e-09   

feature       std                                            ...    abs_diffs  \
channel         0         1         2         3         4    ...            0   
0        0.000019  0.000013  0.000010  0.000008  0.000005    ...     0.007732   
1        0.000021  0.000010  0.000007  0.000003  0.000002    ...     0.008758   
2        0.000022  0.000012  0.000008  0.000004  0.000002    ...     0.009351   
3        0.000020  0.000011  0.000009  0.000005  0.000003    ...     0.008033   
4        0.000019  0.000009  0.000007  0.000005  0.000003    ...     0.007302   
5        0.000022  0.000011  0.000008  0.000004  0.000003    ...     0.009336   
6        0.000024  0.000011  0.000008  0.000004  0.000002    ...     0.010242   
7        0.000020  0.000009  0.000007  0.000003  0.000002    ...     0.008225   
8        0.000024  0.000011  0.000008  0.000004  0.000002    ...     0.009967   
9        0.000021  0.000010  0.000008  0.000005  0.000003    ...     0.008156   
10       0.000023  0.000011  0.000009  0.000005  0.000004    ...     0.009282   
11       0.000020  0.000009  0.000007  0.000004  0.000002    ...     0.008246   
12       0.000022  0.000011  0.000008  0.000004  0.000002    ...     0.008798   
13       0.000023  0.000011  0.000008  0.000004  0.000002    ...     0.009347   
14       0.000019  0.000012  0.000010  0.000008  0.000005    ...     0.007307   
15       0.000018  0.000012  0.000010  0.000008  0.000005    ...     0.007313   

feature                                              skew                      \
channel         1         2         3         4         0         1         2   
0        0.007155  0.011296  0.016871  0.026679  0.053912 -0.349162 -0.268046   
1        0.005656  0.007382  0.008553  0.009507  0.024895 -0.661668 -1.021757   
2        0.006519  0.007910  0.009017  0.010753 -0.093338 -0.616463 -1.447298   
3        0.005679  0.009540  0.011721  0.017858 -0.026314 -0.339795 -0.800418   
4        0.004430  0.008523  0.010877  0.015067  0.022839 -0.412636 -0.337155   
5        0.006264  0.008259  0.010180  0.014606 -0.149228 -0.607120 -1.374854   
6        0.006297  0.008765  0.010048  0.010941  0.069376 -0.458984 -0.689411   
7        0.005071  0.007169  0.008412  0.009543  0.047465 -0.590806 -0.748497   
8        0.006181  0.009430  0.010753  0.011688  0.021843 -0.434773 -0.509331   
9        0.005261  0.009201  0.010691  0.014960  0.001226 -0.286494 -0.605355   
10       0.006169  0.009745  0.012468  0.018744 -0.184541 -0.417096 -0.845515   
11       0.004753  0.007918  